# Center of Gravity for Central Network


# Geo-Locations

The geo-locations of sites can be found in CSV format extracted from SCGX import files. 

In [ ]:
!wget https://raw.githubusercontent.com/robogeekcanada/miscellaneous/main/Geo_location_sites_CSV.csv

--2023-02-24 18:13:58--  https://raw.githubusercontent.com/robogeekcanada/miscellaneous/main/Geo_location_sites_CSV.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8337 (8.1K) [text/plain]
Saving to: ‘Geo_location_sites_CSV.csv’

Geo_location_sites_ 100%[===================>]   8.14K  --.-KB/s    in 0s      

2023-02-24 18:13:58 (58.3 MB/s) - ‘Geo_location_sites_CSV.csv’ saved [8337/8337]



Using Pandas to load the data, display first 10.

In [ ]:
import pandas as pd

data = pd.read_csv(r'Geo_location_sites_CSV.csv')
df = pd.DataFrame(data, columns=['SiteID', 'Name', 'Address', 'City', 'State', 'Country', 'Postal Code', 'Latitude', 'Longitude'])
df[:10].style

,SiteID,Name,Address,City,State,Country,Postal Code,Latitude,Longitude
0,23350,P_CC_Cent_ON_Y001_Brampton,15 Westcreek Blvd.,Brampton,ON,Canada,L6T 5T4,43.684400,-79.702700
1,23351,P_CC_West_AB_Y002_Calgary,3852 23rd Street NE,Calgary,AB,Canada,T2E 6T2,51.086700,-114.006200
2,23352,P_CO_Cent_ON_Y058_YAYA Foods,450b Kipling Avenue,Etobicoke,ON,Canada,M8Z 5E1,43.608500,-79.522100
3,23353,P_CO_Cent_ON_Y059_Joriki,3431 McNicoll Ave,Scarborough,ON,Canada,M1V 2V3,43.821700,-79.256000
4,23354,P_CO_Cent_ON_Y057_Delta Beverages,21 Marycroft Ave,Woodbridge,ON,Canada,L4L 5Y6,43.780600,-79.563500
5,23355,P_CO_Cent_ON_XXXX_CottCentral,0,Mississauga,ON,Canada,L4V 1H6,43.694500,-79.626300
6,23356,P_CO_West_AB_WWWW_CottWest,0,Calgary,AB,Canada,T2C 2V2,50.985650,-113.964100
7,23357,P_CO_Cent_ON_Y060_Alliance,201 Speers Road,Oakville,ON,Canada,L6K 2E8,43.444900,-79.689400
8,23358,P_CO_West_AB_Y061_X-Treme,7910 - 51st Street S.E.,Calgary,AB,Canada,T2C 4R2,50.982900,-113.959800
9,23359,P_CO_West_BC_YC02_LMBCArea,nan,Richmond,BC,Canada,V6V 2B8,49.179250,-123.077200


### Decoding **Name** values

Main challenge **Name** values are composed of a string that determines the type of building, the location type, state/province, site number and site name and full address including latitude and longitude

The **Name** values need to be broken down to help queries later on.

The following functions return the type of site, type of location, the region the site belongs, the site number and the site name.

In [ ]:
def find_type(name):

  if name[0] == 'P':
    return 'Production'
  elif name[0] == 'W':
    return 'Warehouse'

def find_location_type(name):

  if name[2:4] == 'CC':
    return 'Combo Center'
  elif name[2:4] == 'PL':
    return 'Plant'
  elif name[2:4] == 'CO':
    return 'Co-Packer'
  elif name[2:4] == 'PG':
    return 'Purchased Goods'

  elif name[2:4] == 'DB':
    return 'Double Button' 
  elif name[2:4] == 'DC':
    return 'Distribution Center' 
  elif name[2:4] == 'OS':
    return 'Outside Storage' 
  elif name[2:4] == 'XD':
    return 'Cross Dock' 
  elif name[2:4] == 'PT':
    return 'Port' 

def find_region(name):

    if name[5:9] == 'Cent':
      return 'Central'
    elif name[5:9] == 'East':
      return 'East'
    elif name[5:9] == 'West':
      return 'West'
    elif name[5:9] == 'Misc':
      return 'Miscellaneous'

def find_number(name):
  return name[13:17]

def find_site_name(name):
  return name[18:]

### Example of using Decoding **Name** values

In this simple example we sort through the Name of all the sites and decode the first 10.

In [ ]:
for name in df['Name'][:10]:

  print(name + ' is a ' + find_type(name) + ' '+ find_location_type(name) +
        ' located in the ' + find_region(name) + ' region with designation ' + find_number(name)+ " " +
        find_site_name(name))

P_CC_Cent_ON_Y001_Brampton is a Production Combo Center located in the Central region with designation Y001 Brampton
P_CC_West_AB_Y002_Calgary is a Production Combo Center located in the West region with designation Y002 Calgary
P_CO_Cent_ON_Y058_YAYA  Foods is a Production Co-Packer located in the Central region with designation Y058 YAYA  Foods
P_CO_Cent_ON_Y059_Joriki is a Production Co-Packer located in the Central region with designation Y059 Joriki
P_CO_Cent_ON_Y057_Delta Beverages is a Production Co-Packer located in the Central region with designation Y057 Delta Beverages
P_CO_Cent_ON_XXXX_CottCentral is a Production Co-Packer located in the Central region with designation XXXX CottCentral
P_CO_West_AB_WWWW_CottWest is a Production Co-Packer located in the West region with designation WWWW CottWest
P_CO_Cent_ON_Y060_Alliance is a Production Co-Packer located in the Central region with designation Y060 Alliance
P_CO_West_AB_Y061_X-Treme is a Production Co-Packer located in the W

## Adding columns to the Header

The dataframe **df** contains the following columns: *SiteID, Name, Address, City, State, Country, Postal Code, Latitude and Longitude*


In [ ]:
print(df.columns.values)

['SiteID' 'Name' 'Address' 'City' 'State' 'Country' 'Postal Code'
 'Latitude' 'Longitude']


We will add the following columns to the header: *LocationType, CenterType, SiteName, Region and SiteNumber*, these new columns will be inserted after *SiteID*

In [ ]:
header = list(df.columns.values)
header.remove('Name')

extended_info = ['LocationType', 'CenterType', 'SiteName', 'Region', 'SiteNumber']

#insert the extended info to the header
header[1:1] = extended_info
header

['SiteID',
 'LocationType',
 'CenterType',
 'SiteName',
 'Region',
 'SiteNumber',
 'Address',
 'City',
 'State',
 'Country',
 'Postal Code',
 'Latitude',
 'Longitude']

## Extracting data from each data row

### Example

The following example shows how to use **iloc** method to extract the values.

In [ ]:
print(df.iloc[0][0])
print(df.iloc[0][1])

23350
P_CC_Cent_ON_Y001_Brampton


### Finding info for the site

The **find_site_info()** function takes the row index as an input and outputs all the row data.

In [ ]:
def find_site_info(index):

  site_id = df.iloc[index][0]
  location_type = find_type(df.iloc[index][1])
  center_type = find_location_type(df.iloc[index][1])
  site_name = find_site_name(df.iloc[index][1])
  region = find_region(df.iloc[index][1])
  site_number = find_number(df.iloc[index][1])
  address = df.iloc[index][2]
  city = df.iloc[index][3]
  state = df.iloc[index][4]
  country = df.iloc[index][5]
  postal_code = df.iloc[index][6]
  latitude = df.iloc[index][7]
  longitude = df.iloc[index][8]

  site_data = [site_id, location_type, center_type, site_name, region, site_number, address, 
              city, state, country, postal_code, latitude, longitude]
  return site_data

find_site_info(0)

[23350,
 'Production',
 'Combo Center',
 'Brampton',
 'Central',
 'Y001',
 '15 Westcreek Blvd.',
 'Brampton',
 'ON',
 'Canada',
 'L6T 5T4',
 43.6844,
 -79.7027]

### Example

Store all data in an array called **sites_data** by looping through the entire table. In this example we show the first 2 rows.



In [ ]:
sites_data = []

for i in range(len(df.index)):
  sites_data.append(find_site_info(i))

sites_data[:2]

[[23350,
  'Production',
  'Combo Center',
  'Brampton',
  'Central',
  'Y001',
  '15 Westcreek Blvd.',
  'Brampton',
  'ON',
  'Canada',
  'L6T 5T4',
  43.6844,
  -79.7027],
 [23351,
  'Production',
  'Combo Center',
  'Calgary',
  'West',
  'Y002',
  '3852 23rd Street NE',
  'Calgary',
  'AB',
  'Canada',
  'T2E 6T2',
  51.0867,
  -114.0062]]

## Saving CSV file

The **sites_data** array we created previously we will store in a CSV file called **sites_data.csv**. However we first need to add the *header* array we modified earliear.

In [ ]:
import csv

with open('sites_data.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(sites_data)

### Example

Now let's load the **sites_data.csv** to a dataframe called **revised_data**
In this example we will display the first 10.

In [ ]:
revised_data = pd.read_csv(r'sites_data.csv')
revised_data[:10].style

,SiteID,LocationType,CenterType,SiteName,Region,SiteNumber,Address,City,State,Country,Postal Code,Latitude,Longitude
0,23350,Production,Combo Center,Brampton,Central,Y001,15 Westcreek Blvd.,Brampton,ON,Canada,L6T 5T4,43.684400,-79.702700
1,23351,Production,Combo Center,Calgary,West,Y002,3852 23rd Street NE,Calgary,AB,Canada,T2E 6T2,51.086700,-114.006200
2,23352,Production,Co-Packer,YAYA Foods,Central,Y058,450b Kipling Avenue,Etobicoke,ON,Canada,M8Z 5E1,43.608500,-79.522100
3,23353,Production,Co-Packer,Joriki,Central,Y059,3431 McNicoll Ave,Scarborough,ON,Canada,M1V 2V3,43.821700,-79.256000
4,23354,Production,Co-Packer,Delta Beverages,Central,Y057,21 Marycroft Ave,Woodbridge,ON,Canada,L4L 5Y6,43.780600,-79.563500
5,23355,Production,Co-Packer,CottCentral,Central,XXXX,0,Mississauga,ON,Canada,L4V 1H6,43.694500,-79.626300
6,23356,Production,Co-Packer,CottWest,West,WWWW,0,Calgary,AB,Canada,T2C 2V2,50.985650,-113.964100
7,23357,Production,Co-Packer,Alliance,Central,Y060,201 Speers Road,Oakville,ON,Canada,L6K 2E8,43.444900,-79.689400
8,23358,Production,Co-Packer,X-Treme,West,Y061,7910 - 51st Street S.E.,Calgary,AB,Canada,T2C 4R2,50.982900,-113.959800
9,23359,Production,Co-Packer,LMBCArea,West,YC02,nan,Richmond,BC,Canada,V6V 2B8,49.179250,-123.077200


## Plotting a Map

Using **plotly** library to display locations in a map.

For more information on **plotly**: 

https://plotly.com/python-api-reference/generated/plotly.express.scatter_geo.html

https://plotly.com/python/scattermapbox/

In [ ]:
# import the plotly express
import plotly.express as px

# set up the chart from the df dataFrame
fig = px.scatter_geo(revised_data, 
                     # longitude is taken from the revised_data["Longitude"] columns and latitude from revised_data["Latitude"]
                     lon="Longitude", 
                     lat="Latitude", 
                     # choose the map chart's projection
                     projection="natural earth",
                     # columns which is in bold in the pop up
                     hover_name = "SiteName",
                     # format of the popup not to display these columns' data
                     hover_data = {"SiteNumber": True,  
                                   "SiteID":False,
                                   "LocationType": False,
                                   "CenterType": False,
                                   "Region": False,
                                   "Address": False,
                                   "City": False,
                                   "State": False,
                                   "Postal Code": False                                                                    
                                     }
                     )

fig.update_traces(marker=dict(size=5, color="red"))

# fit the map to surround the points
fig.update_geos(fitbounds="locations", showcountries = True)

# add title
fig.update_layout(title = 'Coke Canada sites')
fig.show()

## Using queries

We can perform queries on the dataframe **revised_data**. For example let's look at all the sites that are located in the *Central region*. 

In [ ]:
central_sites = revised_data.query("Region == 'Central'")
central_sites

,SiteID,LocationType,CenterType,SiteName,Region,SiteNumber,Address,City,State,Country,Postal Code,Latitude,Longitude
0,23350,Production,Combo Center,Brampton,Central,Y001,15 Westcreek Blvd.,Brampton,ON,Canada,L6T 5T4,43.684400,-79.702700
2,23352,Production,Co-Packer,YAYA Foods,Central,Y058,450b Kipling Avenue,Etobicoke,ON,Canada,M8Z 5E1,43.608500,-79.522100
3,23353,Production,Co-Packer,Joriki,Central,Y059,3431 McNicoll Ave,Scarborough,ON,Canada,M1V 2V3,43.821700,-79.256000
4,23354,Production,Co-Packer,Delta Beverages,Central,Y057,21 Marycroft Ave,Woodbridge,ON,Canada,L4L 5Y6,43.780600,-79.563500
5,23355,Production,Co-Packer,CottCentral,Central,XXXX,0,Mississauga,ON,Canada,L4V 1H6,43.694500,-79.626300
7,23357,Production,Co-Packer,Alliance,Central,Y060,201 Speers Road,Oakville,ON,Canada,L6K 2E8,43.444900,-79.689400
12,23363,Production,Plant,Weston,Central,Y011,24 Fenmar Dr,Toronto(Weston),ON,Canada,M9L 1L8,43.759800,-79.536800
15,23366,Warehouse,Double Button,Owen Sound,Central,Y008,1795 23rd Street East,Owen Sound,ON,Canada,N4K 5N5,44.586080,-80.914940
16,23367,Warehouse,Double Button,Sault Ste Marie,Central,Y009,987 Great Northern Rd.,Sault Ste Marie,ON,Canada,P6B 0B9,46.547600,-84.327000
17,23368,Warehouse,Cross Dock,Windsor,Central,Y012,3215 Electricity Dr,Windsor,ON,Canada,N8W 5J1,42.284500,-82.965600


### Example

An example of using a function to plot maps from the query using other map settings.

In [ ]:
def display_map(region):

  query_string = 'Region == ' + "'" + region + "'"

  sites = revised_data.query(query_string)

  fig = px.scatter_geo(sites, 
                      # longitude is taken from the revised_data["Longitude"] columns and latitude from revised_data["Latitude"]
                      lon="Longitude", 
                      lat="Latitude", 
                      # choose the map chart's projection
                      projection="natural earth",
                      # columns which is in bold in the pop up
                      hover_name = "SiteName",
                      # format of the popup not to display these columns' data
                      hover_data = {"SiteNumber": True,  
                                    "SiteID":False,
                                    "LocationType": False,
                                    "CenterType": False,
                                    "Region": False,
                                    "Address": False,
                                    "City": False,
                                    "State": False,
                                    "Postal Code": False                                                                    
                                      }
                      )

  fig.update_traces(marker=dict(size=15, color="red"))

  # fit the map to surround the points
  fig.update_geos(fitbounds="locations", showcountries = True)

  fig.update_geos(
    resolution=50,
    showcoastlines=True, coastlinecolor="RebeccaPurple",
    showland=True, landcolor="LightGreen",
    showocean=True, oceancolor="LightBlue",
    showlakes=True, lakecolor="Blue",
    showrivers=True, rivercolor="Blue"
  )


  # add title
  fig.update_layout(title = query_string)
  fig.show()

display_map('Central')

# Understanding Demand

Let's start by downloading the demand from each site and for sourcing sites the volume they produced.

In [ ]:
!wget https://raw.githubusercontent.com/robogeekcanada/miscellaneous/main/demand_CSV.csv

--2023-02-24 18:26:06--  https://raw.githubusercontent.com/robogeekcanada/miscellaneous/main/demand_CSV.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47782 (47K) [text/plain]
Saving to: ‘demand_CSV.csv’

demand_CSV.csv      100%[===================>]  46.66K  --.-KB/s    in 0.007s  

2023-02-24 18:26:06 (6.49 MB/s) - ‘demand_CSV.csv’ saved [47782/47782]



## Demand Data

Let's create a dataframe named **demand_data** and display the first 10 rows.

In [ ]:
demand_data = pd.read_csv(r'demand_CSV.csv')
demand_data[:10].style

,Demand_Location,Demand_Site,Total_Demand,Source_Location,Source_Site,First_Warehouse,First_Stop,Second_Warehouse,Second_Stop,Third_Warehouse,Third_Stop,Product_Details
0,"Y001 Brampton, ON DC",Y001,"23,564,663","Y001 Brampton, ON DC",Y001,nan,nan,nan,nan,nan,nan,"Brampton PET, Core Cans, BIB, 300ml, 250ml, Dasani"
1,"Y001 Brampton, ON DC",Y001,"734,805","Y002 Calgary, AB DC",Y002,"Y071 TCS Calgary, AB DC",Y071,Y053 NFI Toronto Aux WH,Y053,nan,nan,355ml Can Fresca and Diet A&W
2,"Y001 Brampton, ON DC",Y001,"33,868","Y002 Calgary, AB DC",Y002,"Y071 TCS Calgary, AB DC",Y071,Y053 NFI Toronto Aux WH,Y053,nan,nan,1L AHA
3,"Y001 Brampton, ON DC",Y001,"3,414","Y002 Calgary, AB DC",Y002,nan,nan,nan,nan,nan,nan,10L BIB skus
4,"Y001 Brampton, ON DC",Y001,"12,622,135","Y011 Weston, ON DC",Y011,Y053 NFI Toronto Aux WH,Y053,nan,nan,nan,nan,"All Weston can SKUS - hotfill can, monster, non core SSD"
5,"Y001 Brampton, ON DC",Y001,"128,499","Y028 Lachine, QC PC",Y028,Y053 NFI Toronto Aux WH,Y053,nan,nan,nan,nan,500ml PET small runners
6,"Y001 Brampton, ON DC",Y001,"1,883,011",Y057 Delta Beverages,Y057,Y053 NFI Toronto Aux WH,Y053,nan,nan,nan,nan,"Juice, PA, Glass"
7,"Y001 Brampton, ON DC",Y001,"726,501",Y059 Joriki,Y059,Y053 NFI Toronto Aux WH,Y053,nan,nan,nan,nan,VW
8,"Y001 Brampton, ON DC",Y001,"547,957",Y060 Alliance repack,Y060,Y053 NFI Toronto Aux WH,Y053,nan,nan,nan,nan,Variety Packs
9,"Y001 Brampton, ON DC",Y001,"351,905",External Purchases,Exte,nan,nan,nan,nan,nan,nan,"Smartwater, some BIB"


### Example

Pull demand data for Brampton location using a query.

In [ ]:
brampton_volume = demand_data.query("Demand_Site == 'Y001'")
brampton_volume

,Demand_Location,Demand_Site,Total_Demand,Source_Location,Source_Site,First_Warehouse,First_Stop,Second_Warehouse,Second_Stop,Third_Warehouse,Third_Stop,Product_Details
0,"Y001 Brampton, ON DC",Y001,"23,564,663","Y001 Brampton, ON DC",Y001,NaN,NaN,NaN,NaN,NaN,NaN,"Brampton PET, Core Cans, BIB, 300ml, 250ml, Da..."
1,"Y001 Brampton, ON DC",Y001,"734,805","Y002 Calgary, AB DC",Y002,"Y071 TCS Calgary, AB DC",Y071,Y053 NFI Toronto Aux WH,Y053,NaN,NaN,355ml Can Fresca and Diet A&W
2,"Y001 Brampton, ON DC",Y001,"33,868","Y002 Calgary, AB DC",Y002,"Y071 TCS Calgary, AB DC",Y071,Y053 NFI Toronto Aux WH,Y053,NaN,NaN,1L AHA
3,"Y001 Brampton, ON DC",Y001,"3,414","Y002 Calgary, AB DC",Y002,NaN,NaN,NaN,NaN,NaN,NaN,10L BIB skus
4,"Y001 Brampton, ON DC",Y001,"12,622,135","Y011 Weston, ON DC",Y011,Y053 NFI Toronto Aux WH,Y053,NaN,NaN,NaN,NaN,"All Weston can SKUS - hotfill can, monster, no..."
5,"Y001 Brampton, ON DC",Y001,"128,499","Y028 Lachine, QC PC",Y028,Y053 NFI Toronto Aux WH,Y053,NaN,NaN,NaN,NaN,500ml PET small runners
6,"Y001 Brampton, ON DC",Y001,"1,883,011",Y057 Delta Beverages,Y057,Y053 NFI Toronto Aux WH,Y053,NaN,NaN,NaN,NaN,"Juice, PA, Glass"
7,"Y001 Brampton, ON DC",Y001,"726,501",Y059 Joriki,Y059,Y053 NFI Toronto Aux WH,Y053,NaN,NaN,NaN,NaN,VW
8,"Y001 Brampton, ON DC",Y001,"547,957",Y060 Alliance repack,Y060,Y053 NFI Toronto Aux WH,Y053,NaN,NaN,NaN,NaN,Variety Packs
9,"Y001 Brampton, ON DC",Y001,"351,905",External Purchases,Exte,NaN,NaN,NaN,NaN,NaN,NaN,"Smartwater, some BIB"


### Example: Adding Volume

In this example let's take the output of the query and add the total demand.
Notice that string manipulation is required to remove unnecessary commas.

And also a conversion from string > float > int is required to add the demand from each source location

In [ ]:
#brampton_volume[' Total_Demand ']

total_demand = 0

for demand in brampton_volume[' Total_Demand ']:

  demand = demand.strip()
  demand = demand.replace("," , "")

  #print(int(float(demand)))

  total_demand += int(float(demand))

print(f"{total_demand:,}")



41,114,691


### Example: Function to a site volume

In [ ]:
def get_site_demand(site_number):

  query_string = "Demand_Site == " + "'" + site_number + "'"

  site_volume = demand_data.query(query_string)
  return site_volume

In [ ]:
site_demand  = get_site_demand('Y001')

#site_volume  = get_site_volume(site_number)
total_demand = 0

for demand in site_demand[' Total_Demand ']:

  demand = demand.strip()
  demand = demand.replace("," , "")

  #print(int(float(demand)))

  total_demand += int(float(demand))

print(f"{total_demand:,}")

41,114,691


### Example: NFI volume

Since there is no demand from NFI but it's used a first stop, then we can query as follows

In [ ]:
nfi_volume = demand_data.query("First_Stop == 'Y053'")
nfi_volume

,Demand_Location,Demand_Site,Total_Demand,Source_Location,Source_Site,First_Warehouse,First_Stop,Second_Warehouse,Second_Stop,Third_Warehouse,Third_Stop,Product_Details
4,"Y001 Brampton, ON DC",Y001,"12,622,135","Y011 Weston, ON DC",Y011,Y053 NFI Toronto Aux WH,Y053,NaN,NaN,NaN,NaN,"All Weston can SKUS - hotfill can, monster, no..."
5,"Y001 Brampton, ON DC",Y001,"128,499","Y028 Lachine, QC PC",Y028,Y053 NFI Toronto Aux WH,Y053,NaN,NaN,NaN,NaN,500ml PET small runners
6,"Y001 Brampton, ON DC",Y001,"1,883,011",Y057 Delta Beverages,Y057,Y053 NFI Toronto Aux WH,Y053,NaN,NaN,NaN,NaN,"Juice, PA, Glass"
7,"Y001 Brampton, ON DC",Y001,"726,501",Y059 Joriki,Y059,Y053 NFI Toronto Aux WH,Y053,NaN,NaN,NaN,NaN,VW
8,"Y001 Brampton, ON DC",Y001,"547,957",Y060 Alliance repack,Y060,Y053 NFI Toronto Aux WH,Y053,NaN,NaN,NaN,NaN,Variety Packs
...,...,...,...,...,...,...,...,...,...,...,...,...
371,"Y071 TCS Calgary, AB DC",Y071,584,"Y028 Lachine, QC PC",Y028,Y053 NFI Toronto Aux WH,Y053,NaN,NaN,NaN,NaN,small 500ml PET
373,"Y071 TCS Calgary, AB DC",Y071,"240,731",Y057 Delta Beverages,Y057,Y053 NFI Toronto Aux WH,Y053,NaN,NaN,NaN,NaN,"HF Juice, PA"
374,"Y071 TCS Calgary, AB DC",Y071,"94,873",Y059 Joriki,Y059,Y053 NFI Toronto Aux WH,Y053,NaN,NaN,NaN,NaN,VW
375,"Y071 TCS Calgary, AB DC",Y071,"67,040",Y060 Alliance repack,Y060,Y053 NFI Toronto Aux WH,Y053,NaN,NaN,NaN,NaN,Variety Packs


Once we identified all the demand locations that require a first stop in NFI, then we can add this total volume as shown below.

In [ ]:
total_volume = 0

for volume in nfi_volume[' Total_Demand ']:

  volume = volume.strip()
  volume = volume.replace("," , "")

  #print(int(float(demand)))

  total_volume += int(float(volume))

print(f"{total_volume:,}")

69,746,509


### Examples 

The following examples show how to get the total volumes for Weston, Barrie and Central

In [ ]:
def get_sum_volume(site_vol):

  total_volume = 0

  for vol in site_vol[' Total_Demand ']:
    vol = vol.strip()
    vol = vol.replace("," , "")

    total_volume += int(float(vol))
  return total_volume
  #print(f"{total_volume:,}")

  

In [ ]:
weston_volume = get_site_demand('Y011')
get_sum_volume(weston_volume)

615752

In [ ]:
barrie_volume = get_site_demand('Y004')
get_sum_volume(barrie_volume)

8415218

Notice that NFI has zero demand as it's not a DC that distributes but an OS(Outside Storage) location

In [ ]:
central_volume = ['Y001', 'Y011', 'Y004', 'Y005', 'Y006', 'Y007', 'Y010', 'Y024', 'Y053', 'Y054']

total_volume = 0

for volume in central_volume:
  
  vol = get_site_demand(volume)
  sum_vol = get_sum_volume(vol)
  print(f"{volume} {sum_vol :,} ")

  total_volume += sum_vol

print(f"total volume : {total_volume : ,}")

Y001 41,114,691 
Y011 615,752 
Y004 8,415,218 
Y005 8,142,142 
Y006 4,297,028 
Y007 10,456,162 
Y010 2,798,484 
Y024 7,861,575 
Y053 0 
Y054 1,074,646 
total volume :  84,775,698


# Putting all Together

## Example:

In this example we are looking for *(SiteNumber == 'Y001' and LocationType == 'Warehouse')* and find respective *latitude* and *longitude* in float format. Last example removes the empty spaces from *SiteName* values.

In [ ]:
sites_data = pd.read_csv(r'sites_data.csv')

bramptonDC_data = sites_data.query("SiteNumber == 'Y001' and LocationType == 'Warehouse'")

bramptonDC_data

,SiteID,LocationType,CenterType,SiteName,Region,SiteNumber,Address,City,State,Country,Postal Code,Latitude,Longitude
30,23382,Warehouse,Distribution Center,BramptonDC,Central,Y001,15 Westcreek Blvd.,Brampton,ON,Canada,L6T 5T4,43.6844,-79.7027


In [ ]:
lon = float(bramptonDC_data['Longitude'])
lat = float(bramptonDC_data['Latitude'])

print(lat, lon)

43.6844 -79.7027


In [ ]:
str(bramptonDC_data['SiteName']).splitlines()[0][6:]

'BramptonDC'

## Central volumes with respective Lat & Long

In [ ]:
#Excluding Pizza Pizza from Weston

locations = ['Y001', 'Y004', 'Y005', 'Y006', 'Y007', 'Y010', 'Y024', 'Y053', 'Y054']
sites_data = pd.read_csv(r'sites_data.csv')

total_volume = 0

for location in locations:
  
  vol = get_site_demand(location)
  sum_vol = get_sum_volume(vol)

  query_string = "SiteNumber == " + "'" + location + "' " + "and LocationType == 'Warehouse'"
  #print(query_string)
  siteDC_data = sites_data.query(query_string)

  lon = float(siteDC_data['Longitude'])
  lat = float(siteDC_data['Latitude'])
  site_name = str(siteDC_data['SiteName']).splitlines()[0][6:]

  print(f"{volume} {site_name} {sum_vol :,} {lat :} {lon :}")

  total_volume += sum_vol

print('\n')
print(f"total volume : {total_volume : ,}")

Y054 BramptonDC 41,114,691 43.6844 -79.7027
Y054 Barrie 8,415,218 44.3404 -79.6776
Y054 Hamilton 8,142,142 43.2429 -79.8086
Y054 Kingston 4,297,028 44.274197 -76.439718
Y054 London 10,456,162 42.9227 -81.1989
Y054 Sudbury 2,798,484 46.4733 -81.0254
Y054 Ottawa 7,861,575 45.3865 -75.6075
Y054 Toronto 0 43.7481 -79.6204
Y054 Thunder Bay 1,074,646 48.3774 -89.2997


total volume :  84,159,946


# Calculation of MidPoint Center of Gravity

## Summary

The geographic midpoint is calculated by finding the center of gravity for the locations in the 'Your Places' list. The latitude and longitude for each location is converted into Cartesian (x,y,z) coordinates. The x,y, and z coordinates are then multiplied by the weighting factor and added together. A line can be drawn from the center of the earth out to this new x, y, z coordinate, and the point where the line intersects the surface of the earth is the geographic midpoint. This surface point is converted into the latitude and longitude for the midpoint.

Reference: http://www.geomidpoint.com/calculation.html



## Calculations procedure

Given the values for the first location in the list:

Lat1, lon1, years1, months1 and days

1. Convert Lat1 and Lon1 from degrees to radians.

  $lat1 = lat1 * PI/180$

  $lon1 = lon1 * PI/180$

2. Convert lat/lon to Cartesian coordinates for first location.

  $X1 = cos(lat1) * cos(lon1)$

  $Y1 = cos(lat1) * sin(lon1)$
  
  $Z1 = sin(lat1)$

3. Compute weight (by time) for first location.

  $w1= (years1 * 365.25) + (months1 * 30.4375) + days1$

If locations are to be weighted equally, set w1, w2 etc all equal to 1.

4. Repeat steps 1-3 for all remaining locations in the list.

5. Compute combined total weight for all locations.
  $Totweight = w1 + w2 + ... + wn$

6. Compute weighted average x, y and z coordinates.
  $x = ((x1 * w1) + (x2 * w2) + ... + (xn * wn)) / totweight$
  $y = ((y1 * w1) + (y2 * w2) + ... + (yn * wn)) / totweight$
  $z = ((z1 * w1) + (z2 * w2) + ... + (zn * wn)) / totweight$

7. Convert average x, y, z coordinate to latitude and longitude. Note that in Excel and possibly some other applications, the parameters need to be reversed in the atan2 function, for example, use atan2(X,Y) instead of atan2(Y,X).
  
  $Lon = atan2(y, x)$

  $Hyp = sqrt(x * x + y * y)$
  
  $Lat = atan2(z, hyp)$

8. Convert lat and lon to degrees.

  $lat = lat * 180/PI$
  $lon = lon * 180/PI$

9. Special case: If abs(x) < 10-9 and abs(y) < 10-9 and abs(z) < 10-9 then the geographic midpoint is the center of the earth.

## Data upload

The **center_gravity.csv** file includes the volume of NFI. Even though NFI is not a demand site, important to take into account for the Center of Gravity calculation.

In [ ]:
!wget https://raw.githubusercontent.com/robogeekcanada/miscellaneous/main/center_gravity.csv

--2023-02-24 18:58:10--  https://raw.githubusercontent.com/robogeekcanada/miscellaneous/main/center_gravity.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 415 [text/plain]
Saving to: ‘center_gravity.csv.1’

center_gravity.csv. 100%[===================>]     415  --.-KB/s    in 0s      

2023-02-24 18:58:10 (19.7 MB/s) - ‘center_gravity.csv.1’ saved [415/415]



### Data Preview

In [ ]:
sites_data = pd.read_csv(r'center_gravity.csv')
sites_data

,SiteNumber,SiteName,Volume,Latitude,Longitude
0,Y001,BramptonDC,41114691,43.684400,-79.702700
1,Y004,Barrie,8415218,44.340400,-79.677600
2,Y005,Hamilton,8142142,43.242900,-79.808600
3,Y006,Kingston,4297028,44.274197,-76.439718
4,Y007,London,10456162,42.922700,-81.198900
5,Y010,Sudbury,2798484,46.473300,-81.025400
6,Y024,Ottawa,7861575,45.386500,-75.607500
7,Y053,Toronto,69746509,43.748100,-79.620400
8,Y054,Thunder Bay,1074646,48.377400,-89.299700


### Calculations

Working, it could be prettier.

In [ ]:
#Convert to radians and X,Y,Z, steps 1 & 2

import math

x = 0
y = 0
z = 0

X = 0
Y = 0
Z = 0
total_vol = 0

for index in range(len(sites_data.index)):

  site_number = sites_data.iloc[index][0]
  site_name = sites_data.iloc[index][1]
  site_volume = sites_data.iloc[index][2]
  site_latitude = sites_data.iloc[index][3]
  site_longitude = sites_data.iloc[index][4]

  lat_rad = math.radians(site_latitude)
  long_rad = math.radians(site_longitude)

  x1 = math.cos(lat_rad)*math.cos(long_rad)
  y1 = math.cos(lat_rad)*math.sin(long_rad)
  z1 = math.sin(lat_rad)

  total_vol += site_volume
  X += x1*site_volume
  Y += y1*site_volume
  Z += z1*site_volume

  print(f"{site_number :} {site_name :} {site_volume : } {site_latitude :} {site_longitude :} {lat_rad :} {long_rad :} {x1 :} {y1 :} {z1 :}")


# Calculating center of gravity

x = X/total_vol
y = Y/total_vol
z = Z/total_vol

print(total_vol)

print(f"{x} {y} {z}")

Lon = math.atan2(y,x)
Hyp = math.sqrt(x**2 + y**2)
Lat = math.atan2(z, Hyp)

print(Lon, Hyp, Lat)

LAT = math.degrees(Lat)
LON = math.degrees(Lon)

print(LAT, LON)

Y001 BramptonDC  41114691 43.6844 -79.7027 0.7624366117582109 -1.3910745377292844 0.1292682275231787 -0.7115076999726074 0.6906855422207024
Y004 Barrie  8415218 44.3404 -79.6776 0.7738859716512937 -1.3906364600870338 0.12815445555351732 -0.703624623041358 0.6989197560318986
Y005 Hamilton  8142142 43.2429 -79.8086 0.7547309831106559 -1.3929228414071464 0.1288908053600468 -0.7169624232597502 0.6850927265174731
Y006 Kingston  4297028 44.274197 -76.439718 0.7727305113265959 -1.3341247583959748 0.16788097708598454 -0.6960476093390254 0.6980929043229723
Y007 London  10456162 42.9227 -81.1989 0.74914243884577 -1.4171881539976237 0.11204131285450714 -0.7236509556128846 0.6810110415069622
Y010 Sudbury  2798484 46.4733 -81.0254 0.8111120992670808 -1.4141600077454135 0.10743368859022549 -0.6802612733367616 0.7250535170276372
Y024 Ottawa  7861575 45.3865 -75.6075 0.7921438609564063 -1.3195998142016128 0.17457103386500575 -0.6802789583795184 0.7118605853124476
Y053 Toronto  69746509 43.7481 -79.620

### Plotting Map

In [ ]:
fig = px.scatter_geo(sites_data, lon=" Longitude", lat=" Latitude", 
                    projection="natural earth",
                    hover_name = "SiteNumber",
                    hover_data = {" SiteName": True,  
                                  " Volume":True                                                                            
                                    }
                    )

fig.update_traces(marker=dict(size=15, color="red"))

# fit the map to surround the points
fig.update_geos(fitbounds="locations", showcountries = True)

fig.update_geos(
  resolution=50,
  showcoastlines=True, coastlinecolor="RebeccaPurple",
  showland=True, landcolor="LightGreen",
  showocean=True, oceancolor="LightBlue",
  showlakes=True, lakecolor="Blue",
  showrivers=True, rivercolor="Blue"
)


fig.add_scattergeo(lat = [43.86910412214529,0]
                      ,lon = [-79.56321556772812,0]
                      ,hoverinfo = 'none'
                      ,marker_size = 30
                      ,marker_color = 'rgb(65, 105, 225)' # blue
                      ,marker_symbol = 'star'
                      ,showlegend = False
                     )

fig.update_layout(title = 'Center of Gravity for Coke Canada Central Volume')
fig.show()
